In [2]:

import pandas as pd
import numpy as np
from collections import Counter
from copy import copy

In [3]:
man_list = ['a', 'b', 'c', 'd']
women_list = ['A', 'B', 'C', 'D']

In [4]:
women_df = pd.DataFrame({'A': [3,4,2,1], 'B': [3,1,4,2], 'C':[2,3,4,1], 'D':[3,2,1,4]})
women_df.index = man_list

In [5]:
man_df = pd.DataFrame({'A': [1,1,2,4], 'B': [2,4,1,2], 'C':[3,3,3,3], 'D':[4,2,4,1]})
man_df.index = man_list

In [6]:
women_df

,A,B,C,D
a,3,3,2,3
b,4,1,3,2
c,2,4,4,1
d,1,2,1,4


In [7]:
man_df

,A,B,C,D
a,1,2,3,4
b,1,4,3,2
c,2,1,3,4
d,4,2,3,1


In [10]:
#women_df = pd.DataFrame({'A': [3,4,2,1], 'B': [3,1,4,2], 'C':[2,3,4,1], 'D':[3,2,1,4]})
data = [{'A': 0, 'B': 0, 'C':0,'D':0},
        {'A': 1, 'B': 4, 'C':2,'D':3},
       {'A': 3, 'B': 1, 'C':1,'D':2}, 
       {'A': 4, 'B': 3, 'C':4,'D':1}]

# Creates DataFrame.
ranking_df = pd.DataFrame(data)
ranking_df.index = man_list
mentor_list=ranking_df.index.tolist()
ranking_df
print(mentor_list)

['a', 'b', 'c', 'd']


In [12]:
ranking_df
#{('a', 'C'): 3, ('b', 'B'): 4, ('c', 'D'): 2, ('d', 'A'): 4}

#{('b', 'B'): 4, ('c', 'D'): 2, ('d', 'A'): 4}

,A,B,C,D
a,0,0,0,0
b,1,4,2,3
c,3,1,1,2
d,4,3,4,1


In [41]:
# dict to control which women each man can make proposals
women_available = {man:women_list for man in man_list}
print(women_available)
# waiting list of men that were able to create pair on each iteration
waiting_list = []
# dict to store created pairs
proposals = {}
# variable to count number of iterations
count = 0

{'a': ['A', 'B', 'C', 'D'], 'b': ['A', 'B', 'C', 'D'], 'c': ['A', 'B', 'C', 'D'], 'd': ['A', 'B', 'C', 'D']}


In [14]:
#while len(waiting_list)<len(man_list):
waiting_list = []
no_matches=[]
# dict to store created pairs
proposals = {}
count=0
while len(waiting_list)<len(man_list):
    # man makes proposals
    for mentor in mentor_list:
        if mentor not in waiting_list:
           
            """"
            Creates a df with ranking of women that the man can male a proposal too
            A    1
            B    2
            C    3
            D    4
            Then gets the minimum value i.e. the best choice A
            """
            #best_choice = man_df.loc[man][man_df.loc[man].index.isin(women)].idxmin()
            best_choice = ranking_df.loc[mentor].idxmax()
            rank=ranking_df.loc[mentor][best_choice]
            if rank>0:
                # adds to the proposals the man and thier best choice followed by the mans ranking and the womens ranking
                proposals[(mentor, best_choice)]=ranking_df.loc[mentor][best_choice]

                # if women have more than one proposals 
                # she will choose the best option
                overlays = Counter([key[1] for key in proposals.keys()])


                for mentee in overlays.keys():
                    if overlays[mentee]>1:

                        # pairs to drop from proposals
                        # sort by women prefernece
                        pairs={pair: proposals[pair] for pair in proposals.keys() 
                                if mentee in pair}.items()

                        pairs_to_drop=sorted(pairs,key=lambda x: x[1])[:-1]


                        for p_to_drop in pairs_to_drop:
                            # delete proposal
                            del proposals[p_to_drop[0]]
                            ranking_df.at[p_to_drop[0][0],p_to_drop[0][1]]=0
                else:
                    no_matches.append(mentor)
                    
                # man who successfully created pairs must be added to the waiting list 
                waiting_list = [pair[1] for pair in proposals.keys()]+no_matches
                    
    count+=1


In [15]:
proposals

{('b', 'B'): 4, ('c', 'D'): 2, ('d', 'A'): 4}

In [61]:
#while len(waiting_list)<len(man_list):
count=0
while count<10:
    # man makes proposals
    for man in man_list:
        if man not in waiting_list:
            # each man make proposal to the top women from it's list
            women = women_available[man]
            """"
            Creates a df with ranking of women that the man can male a proposal too
            A    1
            B    2
            C    3
            D    4
            Then gets the minimum value i.e. the best choice A
            """
            best_choice = man_df.loc[man][man_df.loc[man].index.isin(women)].idxmin()
            print(man_df.loc[man][best_choice],women_df.loc[man][best_choice])
            # adds to the proposals the man and thier best choice followed by the mans ranking and the womens ranking
            proposals[(man, best_choice)]=(man_df.loc[man][best_choice],women_df.loc[man][best_choice])

            # if women have more than one proposals 
            # she will choose the best option
            overlays = Counter([key[1] for key in proposals.keys()])
            
            print(overlays)
            for women in overlays.keys():
                if overlays[women]>1:
                    
                    # pairs to drop from proposals
                    # sort by women prefernece
                    pairs_to_drop = sorted({pair: proposals[pair] for pair in proposals.keys() 
                            if women in pair}.items(), 
                           key=lambda x: x[1][1]
                          )[1:]
                    print(pairs_to_drop)
                    
                    
                    for p_to_drop in pairs_to_drop:
                        # delete proposal
                        del proposals[p_to_drop[0]]
                        # copy the man's women_aviablabe 
                        _women = copy(women_available[p_to_drop[0][0]])
                        # remove the women from the list
                        _women.remove(p_to_drop[0][1])
                        # update with dropped list
                        women_available[p_to_drop[0][0]] = _women
                    
                    
                    # man who successfully created pairs must be added to the waiting list 
                    waiting_list = [man[0] for man in proposals.keys()]
                    
                    print(proposals)
                    print("Hey")
                    
    count+=1

3 2
Counter({'C': 1})
2 2
Counter({'C': 1, 'D': 1})
2 2
Counter({'C': 1, 'D': 1, 'A': 1})
2 2
Counter({'C': 1, 'D': 1, 'A': 1, 'B': 1})
3 2
Counter({'C': 1, 'D': 1, 'A': 1, 'B': 1})
2 2
Counter({'C': 1, 'D': 1, 'A': 1, 'B': 1})
2 2
Counter({'C': 1, 'D': 1, 'A': 1, 'B': 1})
2 2
Counter({'C': 1, 'D': 1, 'A': 1, 'B': 1})
3 2
Counter({'C': 1, 'D': 1, 'A': 1, 'B': 1})
2 2
Counter({'C': 1, 'D': 1, 'A': 1, 'B': 1})
2 2
Counter({'C': 1, 'D': 1, 'A': 1, 'B': 1})
2 2
Counter({'C': 1, 'D': 1, 'A': 1, 'B': 1})
3 2
Counter({'C': 1, 'D': 1, 'A': 1, 'B': 1})
2 2
Counter({'C': 1, 'D': 1, 'A': 1, 'B': 1})
2 2
Counter({'C': 1, 'D': 1, 'A': 1, 'B': 1})
2 2
Counter({'C': 1, 'D': 1, 'A': 1, 'B': 1})
3 2
Counter({'C': 1, 'D': 1, 'A': 1, 'B': 1})
2 2
Counter({'C': 1, 'D': 1, 'A': 1, 'B': 1})
2 2
Counter({'C': 1, 'D': 1, 'A': 1, 'B': 1})
2 2
Counter({'C': 1, 'D': 1, 'A': 1, 'B': 1})
3 2
Counter({'C': 1, 'D': 1, 'A': 1, 'B': 1})
2 2
Counter({'C': 1, 'D': 1, 'A': 1, 'B': 1})
2 2
Counter({'C': 1, 'D': 1, 'A': 1,

In [43]:
proposals

{('b', 'D'): (2, 2),
 ('d', 'B'): (2, 2),
 ('c', 'A'): (2, 2),
 ('a', 'C'): (3, 2)}

In [ ]:
#if mentees >mentors
    # stop when all mentors are taking up

In [87]:
a_list = ['zero', 'one', 'two', 'three']
a_list[-1]
print(a_list[:-1])

['zero', 'one', 'two']
